In [3]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags

In [4]:
df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\mango_all_game_data_df.parquet")

In [5]:
print(df.columns)
print(df.info())
print(df['source'].value_counts())
df.head()


Index(['source', 'source_path_prefix', 'source_path_suffix', 'length',
       'num_players', 'is_teams', 'player_1_port', 'player_1_character_name',
       'player_1_type_name', 'player_1_stocks', 'player_1_costume',
       'player_1_team_value', 'player_1_ucf_shield_drop_name', 'player_1_tag',
       'player_1_display_name', 'player_2_port', 'player_2_character_name',
       'player_2_type_name', 'player_2_stocks', 'player_2_costume',
       'player_2_team_value', 'player_2_ucf_shield_drop_name', 'player_2_tag',
       'player_2_display_name', 'random_seed', 'slippi', 'stage_name',
       'is_pal', 'is_frozen_ps', 'end_method_name', 'lras_initiator',
       'conclusive', 'winning_player', 'date', 'duration', 'platform',
       'player_1_netplay_code', 'player_1_netplay_name',
       'player_2_netplay_code', 'player_2_netplay_name', 'console_name',
       'all_data_df_common_path', 'inputs_df_common_path',
       'inputs_np_common_path', 'player_1_all_data_df_sub_path',
       'player_

,source,source_path_prefix,source_path_suffix,length,num_players,is_teams,player_1_port,player_1_character_name,player_1_type_name,player_1_stocks,...,player_3_inputs_df_sub_path,player_3_inputs_df_save_path,player_3_inputs_np_sub_path,player_3_inputs_np_save_path,player_4_all_data_df_sub_path,player_4_all_data_df_save_path,player_4_inputs_df_sub_path,player_4_inputs_df_save_path,player_4_inputs_np_sub_path,player_4_inputs_np_save_path
0,mango,D:\Mango\,Slippi-20240312T153007Z-005\Slippi\Game_202208...,130,2,False,0,FALCO,HUMAN,4,...,None,None,None,None,None,None,None,None,None,None
1,mango,D:\Mango\,Slippi-20240312T153007Z-010\Slippi\Game_202101...,405,2,False,0,MEWTWO,HUMAN,4,...,None,None,None,None,None,None,None,None,None,None
2,mango,D:\Mango\,Slippi-20240312T153007Z-009\Slippi\Game_202105...,895,2,False,0,CAPTAIN_FALCON,HUMAN,4,...,None,None,None,None,None,None,None,None,None,None
3,mango,D:\Mango\,Slippi-20240312T153007Z-008\Slippi\Game_202204...,3058,2,False,0,FALCO,HUMAN,4,...,None,None,None,None,None,None,None,None,None,None
4,mango,D:\Mango\,Slippi-20240312T153007Z-005\Slippi\2024-01\Gam...,3625,2,False,0,FOX,HUMAN,4,...,None,None,None,None,None,None,None,None,None,None


In [6]:
filtered_df = df.loc[df['is_teams'] == False]
filtered_df = filtered_df.loc[filtered_df['num_players'] == 2]
filtered_df = filtered_df.loc[filtered_df['player_1_type_name'] == 'HUMAN']
filtered_df = filtered_df.loc[filtered_df['player_2_type_name'] == 'HUMAN']
filtered_df = filtered_df.loc[filtered_df['conclusive'] == True]
filtered_df = filtered_df.loc[filtered_df['source'] == 'mango']
reduced_df = filtered_df[['slippi','source','length','player_1_character_name','player_2_character_name','player_1_display_name','player_2_display_name','player_1_netplay_code','player_2_netplay_code','stage_name','winning_player','player_1_win','player_2_win','player_1_inputs_np_save_path','player_2_inputs_np_save_path']]
# Assuming filtered_df is your DataFrame
# Concatenate the two columns into one Series
all_netplay_codes = pd.concat([filtered_df['player_1_netplay_code'], filtered_df['player_2_netplay_code']])

# Count the occurrences of each netplay code
code_counts = all_netplay_codes.value_counts()

# Display the top 10 most common values
top_10 = code_counts[:25]
print(top_10)
reduced_df.head()

MANG#0      29065
KOD#0        4403
NULL#478     3436
SSJ#998      2961
ZAIN#0       2754
MAGI#732     2657
KJH#23       1672
ALBE#135      777
NEZ#125       644
SALT#747      644
LUCK#0        641
IBDW#0        597
TRT#879       588
MOKY#475      568
LUCK#286      566
SPARK#0       552
WEEP#438      356
SOON#122      279
FFF#641       221
CHUG#596      220
FSGS#282      218
RISH#1        215
IBDW#734      181
VEGG#890      176
ILY#686       153
Name: count, dtype: int64


,slippi,source,length,player_1_character_name,player_2_character_name,player_1_display_name,player_2_display_name,player_1_netplay_code,player_2_netplay_code,stage_name,winning_player,player_1_win,player_2_win,player_1_inputs_np_save_path,player_2_inputs_np_save_path
6,3.9.0,mango,5729,FALCO,FOX,mang,Lookin4Marths,MANG#0,VEGG#890,FINAL_DESTINATION,1.0,True,False,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\7...,C:\Users\jaspa\Grant ML\input_np\mango\FOX\9fe...
7,3.9.0,mango,5877,FALCO,FALCO,mang,TheRealThing,MANG#0,TRT#879,FINAL_DESTINATION,1.0,True,False,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\7...,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\2...
8,3.12.0,mango,6336,MARTH,FALCO,KoDoRiN,mang,KOD#0,MANG#0,POKEMON_STADIUM,1.0,True,False,C:\Users\jaspa\Grant ML\input_np\mango\MARTH\7...,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\1...
10,3.9.0,mango,7744,FOX,CAPTAIN_FALCON,mang,Barry Lyndon,MANG#0,SSJ#998,FOUNTAIN_OF_DREAMS,2.0,False,True,C:\Users\jaspa\Grant ML\input_np\mango\FOX\323...,C:\Users\jaspa\Grant ML\input_np\mango\CAPTAIN...
11,3.9.1,mango,7963,FALCO,FALCO,mang,TheRealThing,MANG#0,TRT#879,FINAL_DESTINATION,1.0,True,False,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\a...,C:\Users\jaspa\Grant ML\input_np\mango\FALCO\2...


In [ ]:
all_data_df = pd.read_parquet(df['player_1_all_data_df_save_path'][0])
print(all_data_df.columns)

In [ ]:
input_np_path = os.path.join(df['inputs_np_common_path'][10],df['player_2_inputs_np_sub_path'][10])
with gzip.open(input_np_path, 'rb') as f:
    input_np = np.load(f)


print(input_np)

In [ ]:
# def process_file(path,time_1,time_2,time_3):
#     save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    
#     time_1_start = time.time()
#     frame_data_df = pd.read_feather(path)
#     time_1.append(time.time()-time_1_start)
    
#     time_2_start = time.time()
#     inputs = np.empty((4,frame_data_df.shape[0]),dtype = float)
#     for i in range(frame_data_df.shape[0]):
#         inputs[0, i] = frame_data_df['pre_joystick_x'][i]
#         inputs[1, i] = frame_data_df['pre_joystick_y'][i]
#         inputs[2, i] = frame_data_df['pre_cstick_x'][i]
#         inputs[3, i] = frame_data_df['pre_cstick_y'][i] 
#         # inputs[4:,i] = one_hot_encode_flags(slp.event.Buttons.Physical,frame_data_df['pre_buttons_physical'][i])
#     time_2.append(time.time()-time_2_start)
    
#     time_3_start = time.time()
#     full_path = os.path.join(save_path, str(uuid.uuid4()) + '.npy.gz')
#     with gzip.open(full_path, 'wb') as f:
#         np.save(f, inputs)
#     time_3.append(time.time() - time_3_start)
#     return inputs         


def process_file(path,time_1,time_2,time_3,buttons):
    save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    time_1_start = time.time()
    frame_data_df = pd.read_parquet(path,engine='pyarrow')
    time_1.append(time.time()-time_1_start)
    num_rows = frame_data_df.shape[0]

    # Assuming one_hot_encode_flags can process a series and return a 2D array
    # encoded_flags = one_hot_encode_flags(slp.event.Buttons.Physical, frame_data_df['pre_buttons_physical'])


    time_2_start = time.time()

    # button_states = frame_data_df['pre_buttons_logical'].to_numpy()[:, None]
    # bitmasks = np.array([button.value for button in buttons if button != buttons.NONE])[None, :]
    # encoded = (button_states & bitmasks) != 0
    # inputs = np.empty((num_rows,25), dtype=float)
    # inputs[ :,0] = frame_data_df['pre_joystick_x']
    # inputs[ :,1] = frame_data_df['pre_joystick_y']
    # inputs[ :,2] = frame_data_df['pre_cstick_x']
    # inputs[ :,2] = frame_data_df['pre_cstick_y']
    # inputs[:,4:] = encoded
    # inputs.T
    
    # Convert the 'pre_buttons_logical' column to a NumPy array
    button_states = frame_data_df['pre_buttons_logical'].values

    # For each button, apply the bitmask and add a column to the DataFrame
    for button in buttons:
        if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
            continue
        # Use broadcasting to apply the bitmask and generate the one-hot encoded array
    frame_data_df[button.name] = (button_states & button.value) != 0
    
    time_2.append(time.time()-time_2_start)
    
    time_3_start = time.time()
    # full_path = os.path.join(save_path, f"{uuid.uuid4()}.npy.gz")
    # with gzip.open(full_path, 'wb') as f:
    #     np.save(f, inputs)
    time_3.append(time.time() - time_3_start)
    
    return 

In [ ]:
# # process_file(path_list[0],1)
# manager = Manager()
# time_1 = manager.list()
# time_2 = manager.list()
# time_3 = manager.list()

# buttons = slp.event.Buttons.Logical
        
# # Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=-1, verbose=1)(delayed(process_file)(path,time_1,time_2,time_3,buttons) for path in tqdm.tqdm(path_list))



In [ ]:
print(sum(time_1))
print(sum(time_2))
print(sum(time_3))


In [ ]:
# frame_data_df =   pd.read_parquet(path_list[1],engine='pyarrow')

# button_states = frame_data_df['pre_buttons_logical'].values

# # For each button, apply the bitmask and add a column to the DataFrame
# for button in buttons:
#     if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
#         continue
#     # Use broadcasting to apply the bitmask and generate the one-hot enco
#     # ed array
#     frame_data_df[button.name] = (button_states & button.value) != 0

In [ ]:

print(pd.read_parquet( df['player_1_inputs_df_save_path'][0],engine='pyarrow').iloc[123:])
with gzip.open(df['player_1_inputs_np_save_path'][0], 'rb') as f:
    input_np = np.load(f)
    


In [ ]:
def np_pd_load_test(df_row,time_1,time_2):
    # path_df  =
    # path_np = 
    
    time_1_start = time.time()
    frame_data_df = pd.read_parquet( df_row['player_1_inputs_df_save_path'],engine='pyarrow')
    time_1.append(time.time()-time_1_start)


    time_2_start = time.time()
    with gzip.open(df_row['player_1_inputs_np_save_path'], 'rb') as f:
        input_np = np.load(f)
    time_2.append(time.time()-time_2_start)
    
    return 

# process_file(path_list[0],1)
manager = Manager()
time_1 = manager.list()
time_2 = manager.list()
        
# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(np_pd_load_test)(df_row,time_1,time_2) for index, df_row in tqdm.tqdm(df.iterrows()))

print(sum(time_1))
print(sum(time_2))